## Week 4 Feature Engineering 

In [1]:
import pandas as pd
import numpy as np

#Read files:
train = pd.read_csv("Store_Train_Forecast_Class.csv")
test = pd.read_csv("Store_Test_Forecast_Class.csv")

In [2]:
train['source']='train'
test['source']='test'
data = pd.concat([train, test],ignore_index=True, sort=True)
print (train.shape, test.shape, data.shape)

(8523, 13) (5681, 12) (14204, 13)


In [13]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Years
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000,14204.000000
mean,141.004977,2181.288914,0.069710,12.792854,1997.830681,15.169319
std,62.086938,1706.499616,0.049728,4.652502,8.371664,8.371664
min,31.290000,33.290000,0.003575,4.555000,1985.000000,4.000000
25%,94.012000,834.247400,0.031145,8.710000,1987.000000,9.000000
50%,142.247000,1794.331000,0.057194,12.600000,1999.000000,14.000000
75%,185.855600,3101.296400,0.096930,16.750000,2004.000000,26.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000,28.000000


In [3]:
# Shall we combine outlets Supermarket Type2 and Type3

data.pivot_table(values='Item_Outlet_Sales',index='Outlet_Type')

,Item_Outlet_Sales
Outlet_Type,
Grocery Store,339.828500
Supermarket Type1,2316.181148
Supermarket Type2,1995.498739
Supermarket Type3,3694.038558


In [4]:
# item visibilty

#Determine average visibility of a product
visibility_avg = data.pivot_table(values='Item_Visibility', index='Item_Identifier')

#Impute 0 values with mean visibility of that product:
miss_bool = (data['Item_Visibility'] == 0)

print ('Number of 0 values initially: %d'%sum(miss_bool) )
#data.loc[miss_bool,'Item_Weight']  = data.loc[miss_bool,'Item_Identifier']
#.apply(lambda x: item_avg_weight.at[x,'Item_Weight'])
#data.loc[miss_bool,'Item_Visibility'] = data.loc[miss_bool,'Item_Identifier']
#.apply(lambda x: visibility_avg[x])
data.loc[miss_bool,'Item_Visibility'] = data.loc[miss_bool,'Item_Identifier']
.apply(lambda x: visibility_avg.at[x,'Item_Visibility'])
print ('Number of 0 values after modification: %d'%sum(data['Item_Visibility'] == 0) )


Number of 0 values initially: 879
Number of 0 values after modification: 0


In [5]:
# Create a new broader category
# If you look at the Item_Identifier, i.e. the unique ID of each item, it starts with either FD, DR or NC. 
# If you see the categories, these look like being Food, Drinks and Non-Consumables. 
# So I’ve used the Item_Identifier variable to create a new column:

In [6]:
#Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [7]:
#Age of the store in Years:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

In [8]:
#Change categories of low fat:
print ('Original Categories:')
print (data['Item_Fat_Content'].value_counts() )

print ('\nModified Categories:')
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print (data['Item_Fat_Content'].value_counts())

Original Categories:
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Modified Categories:
Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64


In [10]:
#Mark non-consumables as separate category in low_fat:
data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
data['Item_Fat_Content'].value_counts()

Low Fat       6499
Regular       5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64

In [14]:
#Age of the store in Years:
data['Price_Per_Weight'] = data['Item_MRP'] / data['Item_Weight']
data['Price_Per_Weight'].describe()

count    11765.000000
mean        12.801009
std          8.059203
min          1.665236
25%          6.840929
50%         10.984840
75%         16.356724
max         50.775731
Name: Price_Per_Weight, dtype: float64

In [15]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Years,Price_Per_Weight
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000,14204.000000,11765.000000
mean,141.004977,2181.288914,0.069710,12.792854,1997.830681,15.169319,12.801009
std,62.086938,1706.499616,0.049728,4.652502,8.371664,8.371664,8.059203
min,31.290000,33.290000,0.003575,4.555000,1985.000000,4.000000,1.665236
25%,94.012000,834.247400,0.031145,8.710000,1987.000000,9.000000,6.840929
50%,142.247000,1794.331000,0.057194,12.600000,1999.000000,14.000000,10.984840
75%,185.855600,3101.296400,0.096930,16.750000,2004.000000,26.000000,16.356724
max,266.888400,13086.964800,0.328391,21.350000,2009.000000,28.000000,50.775731
